In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [26]:
model_type="knn"
pca_dims=5

In [32]:
def load_data(filename):
    data = np.load(filename)
    X=data[:,0:-1]
    y=data[:,-1]
    return X,y

def standardize(image_data,axis=1):
    # Compute mean and std along axis 1, assuming each image is a row
    mean = np.mean(image_data, axis=axis, keepdims=True)
    std = np.std(image_data, axis=axis, keepdims=True)
    # Normalize each image
    scaled_images = (image_data - mean) / std
    return scaled_images

def scale(image_data):
    return standardize(image_data,axis=1)
    return standardize(standardize(image_data,axis=1),axis=0)

X, y = load_data("data/fashion_train.npy")
# train and test model in a loop
n=10
cms=[]
accs=[]
precs=[]
recalls=[]
for i in range(n):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,stratify=y)
    
    #scale data
    X_train_scaled=scale(X_train)

    # perform PCA
    pca = PCA(n_components=pca_dims,random_state=42)
    pca.fit(X_train_scaled)
    X_train_pca=pca.transform(X_train_scaled)

    #fit model of choice
    if model_type=="knn":
        #create KNN
        model=KNeighborsClassifier(n_neighbors=20)
    elif model_type=="mlp":
        #create MLP
        model = MLPClassifier(solver='lbfgs', alpha=0.05,hidden_layer_sizes=(25,5, 2), random_state=1,max_iter=10_000)

    model.fit(X_train_pca,y_train)

    X_test_scaled=scale(X_test)
    X_test_pca=pca.transform(X_test_scaled)

    #create predictions with model
    preds = model.predict(X_test_pca)
    acc = metrics.accuracy_score(y_test,preds)
    cm = metrics.confusion_matrix(y_test,preds)
    prec=metrics.precision_score(y_test,preds,average=None)
    recall=metrics.recall_score(y_test,preds,average=None)
    
    cms.append(cm)
    accs.append(acc)
    precs.append(prec)
    recalls.append(recall)

print(f"Accuracy: {np.average(accs)} +- {np.std(accs)}")

Accuracy: 0.79356 +- 0.006465786881733724
